In [3]:
pip install mistralai==1.0.0

SyntaxError: invalid syntax (3787403300.py, line 1)

In [3]:
import os
from mistralai import Mistral
import re

api_key = os.environ["j9BzEXRhSmEdQjf9VNpjYrLlrCR73hge"]

client = Mistral(api_key=api_key)


ModuleNotFoundError: No module named 'mistralai'

In [ ]:
agent_id= "ag:8e2706f0:20240807:python-agent:ffdc79f8"

In [ ]:
def run_python_agent(query):
    """
    Sends a user query to a Python agent and returns the response.

    Args:
        query (str): The user query to be sent to the Python agent.

    Returns:
        str: The response content from the Python agent.
    """
    print("### Run Python agent")
    print(f"User query: {query}")
    try:
        response = client.agents.complete(
            agent_id= agent_id,
            messages = [
                {
                    "role": "user",
                    "content":  query
                },
            ]
        )
        result = response.choices[0].message.content
        return result
    except Exception as e:
        print(f"Request failed: {e}. Please check your request.")
        return None

def extract_pattern(text, pattern):
    """
    Extracts a pattern from the given text.

    Args:
        text (str): The text to search within.
        pattern (str): The regex pattern to search for.

    Returns:
        str: The extracted pattern or None if not found.
    """
    match = re.search(pattern, text, flags=re.DOTALL)
    if match:
        return match.group(1).strip()
    return None

def extract_code(result):
    """
    Extracts Python function and test case from the response content.

    Args:
        result (str): The response content from the Python agent.

    Returns:
        tuple: A tuple containing the extracted Python function, test function, and a retry flag.
    """
    retry = False
    print("### Extracting Python code")
    python_function = extract_pattern(result, r'## Python function(.*?)## Test case')
    if not python_function:
        retry = True
        print("Python function failed to generate or wrong output format. Setting retry to True.")

    print("### Extracting test case")
    test_function = extract_pattern(result, r'## Test case(.*?)```')
    if not test_function:
        retry = True
        print("Test function failed to generate or wrong output format. Setting retry to True.")

    return python_function, test_function, retry

def check_code(python_function, test_function):
    """
    Executes the Python function and its test case, and checks for any errors.

    Args:
        python_function (str): The Python function to be executed.
        test_function (str): The test case to be executed.

    Returns:
        bool: A flag indicating whether the code execution needs to be retried.

    Warning:
        This code is designed to run code that’s been generated by a model, which may not be entirely reliable.
        It's strongly recommended to run this in a sandbox environment.
    """
    retry = False
    try:
        exec(python_function)
        print("Code executed successfully.")
        try:
            exec(test_function)
            print("Code passed test case.")
        except Exception:
            print(f"Test failed")
            retry = True
            print("Setting retry to True")
    except Exception:
        print(f"Code failed.")
        retry = True
        print("Setting retry to True")
    return retry

def run_workflow(query):
    """
    Runs the complete workflow to generate, extract, and validate Python code.

    Args:
        query (str): The user query to be processed.
    """
    print("### ENTER WORKFLOW")
    i = 0
    max_retries = 3
    retry = True # just to get it started
    while i < max_retries and retry:
        print(f"TRY # {i}")
        i += 1
        result = run_python_agent(query)
        python_function, test_function, retry = extract_code(result)
        retry = check_code(python_function, test_function)

    if not retry:
        print(f"Validated Python function: ```{python_function}```")
    print("### EXIT WORKFLOW")



In [ ]:
run_workflow("How can I remove duplicates from a list")